# Optimising the regularisation parameter
--- 
Running ppxf with regularisation is very time-intensive and makes it difficult to run simulations across a large parameter space. 

Up until now we've been using $\Delta\chi^2 < 1$ as our condition for terminating regularisation, and an upper limit of `regul` = 50,000. 
* **Can we find a better maximum value for `regul`?** How does the solution change between 50,000 and e.g. 100,000? If there is no change, does this hold for SFHs of all shapes?
* **Can we find a less strict convergence criterion?** How does a solution with $\Delta\chi^2 < 1$ differ from a solution with $\Delta\chi^2 \sim 10$, for instance? Is there a meaningful difference?  

In [1]:
%matplotlib widget

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [3]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
from itertools import product
import multiprocessing
import pandas as pd
import extinction

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, compute_mw_age, compute_lw_age, compute_cumulative_mass, compute_cumulative_light
from ppxftests.sfhutils import compute_mean_age, compute_mean_mass, compute_mean_sfr, compute_mean_1D_sfh
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted, ppxf_plot
from ppxftests.plot_ppxf_summary import plot_ppxf_summary

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"
data_path = "/priv/meggs3/u5708159/ppxftests/"


In [4]:
##################################################################
# Settings
##################################################################
isochrones = "Padova"
SNR = 100
z = 0
niters = 20
nthreads = 20


In [5]:
##################################################################
# Define the input SFH
##################################################################
gal = 10
sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_star_kms = load_sfh(gal, plotit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
###############################################################################
# Create spectrum
###############################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# Run ppxf with a fixed regul value
pp_list = []
for regul in tqdm(regul_vals):
    pp = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1, mdegree=4, reddening=None,
                  regularisation_method="none", regul=regul,
                  fit_agn_cont=False,
                  isochrones=isochrones,
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    pp_list.append(pp)


KeyboardInterrupt: 

In [21]:
###########################################################################
# Helper function for running MC simulations
###########################################################################
def ppxf_helper(regul):
    # Run ppxf
    pp = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1, mdegree=4, reddening=None,
                  regularisation_method="none", regul=regul, 
                  fit_agn_cont=False,
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


In [57]:
# Input arguments
regul_vals = np.logspace(-1, 7, 9)
regul_vals[0] = 0
args_list = regul_vals

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
nthreads = len(regul_vals)
niters = len(regul_vals)
with multiprocessing.Pool(nthreads) as pool:
    pp_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
print(f"Total time in ppxf: {time() - t:.2f} s")


Running ppxf on 8 threads...



Total time in ppxf: 35.59 s


In [58]:
# We're only really interested in the convergence of the solution (not with how accurate it is)
# So just plot the 1D SFH
fig = plt.figure(figsize=(10, 3.5))
ax = fig.add_axes([0.1, 0.25, 0.8, 0.65])

# Plot the SFH
cmap_regul = matplotlib.cm.get_cmap("Spectral", len(regul_vals))
for ii, pp in enumerate(pp_mc_list):
    ax.step(pp.ages, np.log10(pp.sfh_mw_1D), where="mid", color=cmap_regul(ii), label=f"regul = {pp.regul}")   

# Decorations
ax.set_ylabel(r"Stellar mass ($\rm M_\odot$)")
ax.set_xlabel("Age (yr)")
ax.set_xscale("log")
ax.grid()
ax.legend(fontsize="small")
ax.autoscale(axis="x", tight=True, enable=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
# Plot the sum of the absolute difference in the SFH (in M_sun) expressed as a % of the total stellar mass 
delta_M = np.array([np.nansum(np.abs(pp_list[ii].sfh_mw_1D - pp_list[ii - 1].sfh_mw_1D)) for ii in range(1, len(pp_list))])
M_tot_vals = np.array([np.nansum(pp.sfh_mw_1D) for pp in pp_list])

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(regul_vals[1:], delta_M / M_tot_vals[1:], marker="D")
ax.grid()
ax.set_xlabel("regul")
ax.set_ylabel(r"$|\Delta M|/M_{\rm tot} \,\rm (M_\odot)$")
ax.set_xscale("log")
ax.set_yscale("log")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [60]:
# How much does the mass/light-weighted age change as a function of regul?
age_thresh = 1e9
log_mw_age_list = [compute_mw_age(pp.sfh_mw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_list]
log_lw_age_list = [compute_lw_age(pp.sfh_lw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_list]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(regul_vals, log_mw_age_list, marker="D", label=f"Mass-weighted age (< {age_thresh / 1e6:.1f} Myr)")
ax.plot(regul_vals, log_lw_age_list, marker="D", label=f"Light-weighted age (< {age_thresh / 1e6:.1f} Myr)")
ax.grid()
ax.set_xlabel("regul")
ax.set_ylabel("Mean age")
ax.set_xscale("log")
ax.legend()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Remarks 
---
The MW/LW mean age and the SFH actually continue to change beyond our previously assumed cutoff-value of 5e4. **We therefore need to increase the maximum regularsiation parameter.** 

## How small does $\Delta \chi^2 - \Delta \chi^2_{\rm ideal}$ need to be to achieve a satisfactory level of convergence?
---
2 things to test here:
* how much does the optimal value of `regul` change between identical runs? Run ppxf w/ automatic regularisation & compare the SFHs and other derived parameters between runs. This will tell us whether or not we need to re-run ppxf multiple times (like an MC simulation) to get reliable results. 
* how much do the results change for $\Delta \chi^2 - \Delta \chi^2_{\rm ideal} < 1$ vs. $\Delta \chi^2 - \Delta \chi^2_{\rm ideal} = 10$? If it's not a big difference then we can revise `delta_regul_min`.

In [ ]:
########################################################################
# How much does the optimal value of regul change between runs?
########################################################################
pp_opt_list = []
for ii in range(10):
    pp_opt = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                      z=z, ngascomponents=1, mdegree=-1, reddening=None,
                      regularisation_method="auto", regul_start=1.5e4, regul_max=10e4,
                      fit_agn_cont=False, fit_gas=False, tie_balmer=False,
                      isochrones="Padova")
    pp_opt_list.append(pp_opt)

In [ ]:
########################################################################
# Print the regul value from each run
########################################################################
for ii, pp in enumerate(pp_opt_list):
    print(f"Iteration {ii}: Delta-delta-chi2 = {pp.delta_delta_chi2:.4f}, regul_opt = {pp.regul:.1f}")

########################################################################
# How much does the SFH change?
########################################################################
fig = plt.figure(figsize=(10, 3.5))
ax = fig.add_axes([0.1, 0.25, 0.8, 0.65])

# Plot the SFH
cmap_regul = matplotlib.cm.get_cmap("Spectral", len(pp_opt_list))
for ii, pp in enumerate(pp_opt_list):
    ax.step(pp.ages, np.log10(pp.sfh_mw_1D), where="mid", color=cmap_regul(ii), label=f"regul = {pp.regul}")   

# Decorations
ax.set_ylabel(r"Log stellar mass ($\rm \log_{10}\, M_\odot$)")
ax.set_xlabel("Age (yr)")
ax.set_xscale("log")
ax.grid()
ax.legend(fontsize="small")
ax.autoscale(axis="x", tight=True, enable=True)

########################################################################
# How much does the LW/MW age change?
########################################################################
age_thresh = 1e9
log_mw_age_list = [compute_mw_age(pp.sfh_mw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_opt_list]
log_lw_age_list = [compute_lw_age(pp.sfh_lw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_opt_list]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(range(len(pp_opt_list)), log_mw_age_list, marker="D", label=f"Mass-weighted age (< {age_thresh / 1e6:.1f} Myr)")
ax.plot(range(len(pp_opt_list)), log_lw_age_list, marker="D", label=f"Light-weighted age (< {age_thresh / 1e6:.1f} Myr)")
ax.grid()
ax.set_xlabel("Iteration")
ax.set_ylabel("Mean age")
ax.set_xscale("log")
ax.legend()

----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 7.04 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3566...
Iteration 1: Regularisation parameter range: 15000.0-25000.0 (n = 21)
Iteration 1: Running ppxf on 20 threads...


In [89]:
################################################################################################
# How much do the SFH, mean age etc. change in a small window around the optimal regul value?
################################################################################################

# Galaxy 10: regl = 5e4 has a reasonably small delta-delta chi2
# for regul in [5e4 - 100, 5e4, 5e4 + 100]
pp = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1, mdegree=-1, reddening=None,
              regularisation_method="fixed", regul_fixed=1.4e4, 
              fit_agn_cont=False, fit_gas=False, tie_balmer=True,
              isochrones="Padova")


----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 1.69 s
Iteration 1: Scaling noise by 1.3533...
Best Fit:       Vel     sigma
 comp. 0:        14       117
chi2/DOF: 1.021
method = capfit ; Jac calls: 3 ; Func calls: 11 ; Status: 4
Nonzero Templates:  204  /  222
----------------------------------------------------
Desired Delta Chi^2: 95.08
Current Delta Chi^2: 97.18
Delta-Delta Chi^2: 2.1
----------------------------------------------------
Elapsed time in PPXF: 6.57 s


In [90]:
pp.delta_delta_chi2

2.1003604580024557

95.0789145920377